In [42]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [6]:
# Importação de dados de teste e treino

test = pd.read_csv(r"C:\Users\laila\Documents\ONCASE\Cientista de dados\classification_data\classification_test.csv")
train = pd.read_csv(r"C:\Users\laila\Documents\ONCASE\Cientista de dados\classification_data\classification_train.csv")

train.head()

,x1,x2,target
0,1.934224,-0.111822,1
1,1.767383,-0.976402,1
2,0.937101,-0.085633,1
3,1.340732,0.166960,1
4,-0.199901,-0.155204,1


In [21]:
# Definindo entradas e saídas

# Treinamento

X_train = train.iloc[:, 0:2]
Y_train = train.iloc[:, 2]

# Teste

X_test = test.iloc[:, 0:2]
Y_test = test.iloc[:, 2]


In [40]:
# Criação, compilação, fitting e avaliação do modelo

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(2,)),
    keras.layers.Dense(4, activation=tf.nn.relu),
    keras.layers.Dense(4, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])


model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall()])
model.fit(X_train, Y_train, epochs=30, batch_size=1)
test_loss, test_acc, test_prec, test_rec = model.evaluate(X_test, Y_test)
test_f2score = 5*(test_prec*test_rec)/((4*test_prec) + test_rec)

# CRIAR GRÁFICO PRA LOSS, ACCURACY, PRECISION, RECALL
plt.plot(list(range(30)), )
plt.show()

print('Test accuracy: {:.4f}\nTest precision: {:.4f}\nTest recall: {:.4f}\nTest f2-score: {:.4f}'.format(
    test_acc, test_prec, test_rec, test_f2score))

Epoch 1/30
670/670 [==============================] - 1s 950us/step - loss: 0.2276 - accuracy: 0.6552 - precision_6: 0.7213 - recall_6: 0.5192
Epoch 2/30
670/670 [==============================] - 1s 930us/step - loss: 0.1713 - accuracy: 0.8000 - precision_6: 0.8547 - recall_6: 0.7286
Epoch 3/30
670/670 [==============================] - 1s 951us/step - loss: 0.1407 - accuracy: 0.8313 - precision_6: 0.8266 - recall_6: 0.8437
Epoch 4/30
670/670 [==============================] - 1s 944us/step - loss: 0.1292 - accuracy: 0.8299 - precision_6: 0.8099 - recall_6: 0.8673
Epoch 5/30
670/670 [==============================] - 1s 921us/step - loss: 0.1240 - accuracy: 0.8403 - precision_6: 0.8258 - recall_6: 0.8673
Epoch 6/30
670/670 [==============================] - 1s 941us/step - loss: 0.1205 - accuracy: 0.8403 - precision_6: 0.8258 - recall_6: 0.8673
Epoch 7/30
670/670 [==============================] - 1s 929us/step - loss: 0.1182 - accuracy: 0.8448 - precision_6: 0.8273 - recall_6: 0.8761